# Prova Intermediária - Machine Learning

**Nome do aluno**: <font color='red'>Escreva seu nome aqui</font>

Você deve submeter o trabalho via Blackboard. É de sua responsabilidade garantir que o arquivo correto foi enviado.

Se você precisou adicionar arquivos extras à essa prova, assegure-se de que você os está enviando também.

A prova é individual e com consulta. Faça o seu trabalho de maneira ética! Se você recebeu ajuda de sistemas de apoio inteligentes (por exemplo, ChatGPT, Bard, CoPilot, etc) DECLARE CLARAMENTE ONDE VOCÊ RECEBEU AJUDA para não configurar plágio (definido como "declarar como próprio o trabalho construido por outrem - neste caso um sistema computacional")

São 10 questões, cada uma valendo 1 ponto.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

np.random.seed(42)

# Contexto do problema

Vamos trabalhar com um *dataset* de processadores antigos, da década de 1980! Nesses tempos o *hardware* era muito mais simples que nos dias atuais. O *dataset* possui atributos referentes ao *hardware* de diferentes máquinas, e o *target* é uma métrica de desempenho computacional relativo (relativo a qual valor-base? Honestamente, não sei, mas paciência).

Aqui está a documentação original do *dataset*:


```
1. Title: Relative CPU Performance Data 

2. Source Information
   -- Creators: Phillip Ein-Dor and Jacob Feldmesser
     -- Ein-Dor: Faculty of Management; Tel Aviv University; Ramat-Aviv; 
        Tel Aviv, 69978; Israel
   -- Donor: David W. Aha (aha@ics.uci.edu) (714) 856-8779   
   -- Date: October, 1987
 
3. Past Usage:
    1. Ein-Dor and Feldmesser (CACM 4/87, pp 308-317)
       -- Results: 
          -- linear regression prediction of relative cpu performance
          -- Recorded 34% average deviation from actual values 
    2. Kibler,D. & Aha,D. (1988).  Instance-Based Prediction of
       Real-Valued Attributes.  In Proceedings of the CSCSI (Canadian
       AI) Conference.
       -- Results:
          -- instance-based prediction of relative cpu performance
          -- similar results; no transformations required
    - Predicted attribute: cpu relative performance (numeric)

4. Relevant Information:
   -- The estimated relative performance values were estimated by the authors
      using a linear regression method.  See their article (pp 308-313) for
      more details on how the relative performance values were set.

5. Number of Instances: 209 

6. Number of Attributes: 10 (6 predictive attributes, 2 non-predictive, 
                             1 goal field, and the linear regression's guess)

7. Attribute Information:
   1. vendor name: 30 
      (adviser, amdahl,apollo, basf, bti, burroughs, c.r.d, cambex, cdc, dec, 
       dg, formation, four-phase, gould, honeywell, hp, ibm, ipl, magnuson, 
       microdata, nas, ncr, nixdorf, perkin-elmer, prime, siemens, sperry, 
       sratus, wang)
   2. Model Name: many unique symbols
   3. MYCT: machine cycle time in nanoseconds (integer)
   4. MMIN: minimum main memory in kilobytes (integer)
   5. MMAX: maximum main memory in kilobytes (integer)
   6. CACH: cache memory in kilobytes (integer)
   7. CHMIN: minimum channels in units (integer)
   8. CHMAX: maximum channels in units (integer)
   9. PRP: published relative performance (integer)
  10. ERP: estimated relative performance from the original article (integer)

8. Missing Attribute Values: None

9. Class Distribution: the class value (PRP) is continuously valued.
   PRP Value Range:   Number of Instances in Range:
   0-20               31
   21-100             121
   101-200            27
   201-300            13
   301-400            7
   401-500            4
   501-600            2
   above 600          4

Summary Statistics:
	   Min  Max   Mean    SD      PRP Correlation
   MCYT:   17   1500  203.8   260.3   -0.3071
   MMIN:   64   32000 2868.0  3878.7   0.7949
   MMAX:   64   64000 11796.1 11726.6  0.8630
   CACH:   0    256   25.2    40.6     0.6626
   CHMIN:  0    52    4.7     6.8      0.6089
   CHMAX:  0    176   18.2    26.0     0.6052
   PRP:    6    1150  105.6   160.8    1.0000
   ERP:   15    1238  99.3    154.8    0.9665

```

Para melhor entender este *dataset*, segue uma explicação de cada atributo:

- `vendor name`: Nome do fabricante do processador.
- `Model Name`: Nome do modelo do processador.
- `MYCT`: Duração do ciclo de *clock* deste processador, em nanossegundos. Nesta época as frequências de trabalho das CPUs eram medidas em kHz ou MHz, e portando as durações de ciclo eram da ordem de magnitude de centenas de nanossegundos.
- `MMIN`, `MMAX`: Faixa de variação permitida para a quantidade de memória RAM aceita por esse processador.
- `CACH`: Tamanho da memória *cache* deste processador. Note que alguns processadores nem tinham memória *cache*.
- `CHMIN`, `CHMAX`: Faixa de valores de quantidade de canais de comunicação deste processador com periféricos.
- `PRP`: Performance relativa. <font color="red">Este é o *target*</font>
- `ERP`: Performance estimada pelos autores do artigo citado acima - <font color="red">Não deve ser usado como *feature*!</font>


# Análise exploratória

## Lendo o dataset

In [ ]:
column_names = [
    'Vendor',
    'Model',
    'MYCT',
    'MMIN',
    'MMAX',
    'CACH',
    'CHMIN',
    'CHMAX',
    'PRP',
    'ERP',
]

df = pd.read_csv('machine.data', names=column_names)


In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
from sklearn.utils import shuffle

df = shuffle(df)
df

## Preparação inicial do *dataset*

### `Vendor`

In [ ]:
vendor_counts = df['Vendor'].value_counts()
vendor_counts


In [ ]:
ax = vendor_counts \
    .sort_values(ascending=True) \
    .plot \
    .barh(figsize=(10, 10))
ax.bar_label(ax.containers[0])
plt.tight_layout()
plt.show()

Tem muitos fabricantes com poucos modelos de CPU. Vamos agrupá-los em uma categoria `others`:

In [ ]:
small_vendors = list(vendor_counts[vendor_counts <= 10].index)
small_vendors

In [ ]:
df.loc[df['Vendor'].isin(small_vendors), 'Vendor'] = 'others'

In [ ]:
vendor_counts = df['Vendor'].value_counts()
vendor_counts


In [ ]:
ax = vendor_counts \
    .sort_values(ascending=True) \
    .plot \
    .barh(figsize=(10, 5))
ax.bar_label(ax.containers[0])
plt.tight_layout()
plt.show()


In [ ]:
df['Vendor'] = df['Vendor'].astype('category')

In [ ]:
df.info()

### `Model`

O nome do modelo não significa nada, vamos descartar essa coluna:

In [ ]:
df = df.drop(columns=['Model'])

## Estatísticas descritivas simples e visualizações

As estatísticas descritivas simples (contagens, médias, medianas, desvios, etc.) são a primeira verificação do dataset, para que tenhamos uma noção da natureza dos dados. Tendo esse entendimento inicial, teremos melhor capacidade de conceber boas visualizações dos dados para continuar a ampliar nosso entendimento da situação.

In [ ]:
import warnings


def visualize(df):
    print('Estatísticas descritivas das variáveis contínuas')
    print(df.describe(include=np.number))
    print()

    print('Estatísticas descritivas das variáveis categóricas')
    print(df.describe(include='category'))
    print()

    df_train_numericals = df.select_dtypes(np.number)

    print('Correlações entre as variáveis numéricas')
    print(df_train_numericals.corr())
    print()

    print('Visualizações')

    pd.plotting.scatter_matrix(df_train_numericals, figsize=(12, 12))
    plt.tight_layout()
    plt.show()

    with warnings.catch_warnings(action='ignore', category=FutureWarning):
        df.plot.box(
            by='Vendor',
            layout=(4, 2),
            figsize=(8, 16),
            xlabel='Model',
            rot=90,
        )
    plt.tight_layout()
    plt.show()


visualize(df)


## One-hot-encoding, e separando *features* e *target*

Lembre-se que a coluna `ERP` não é nem *feature*, nem *target*: trata-se dos valores preditos de desempenho conforme descrito no artigo dos autores do *dataset*. Não vamos usar essa coluna no modelo.

In [ ]:
df = pd.get_dummies(df)
df.head()

In [ ]:
X = df.drop(columns=['PRP', 'ERP', 'Vendor_others']).copy()
y = df['PRP'].copy()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

# Prova intermediária

## Parte prática

Ao longo das questões seguintes você vai construir um projeto de regressão.

**Questão 1**: Analise o desempenho RMSE de um modelo de regressão linear para estes dados usando validação cruzada com 5 partições.

Dica: use a métrica `"neg_mean_squared_error"` no argumento `scoring` da função `cross_val_score`, e corrija adequadamente os escores obtidos.

**Questão 2**: Construa uma função que recebe a tabela de *features* (i.e. `X_train` ou `X_test`) e retorna os dados com *feature engineering* conforme descrito a seguir:

- Primeiro transforme a coluna `MYCT`: aplique uma transformação do tipo $f(x) = 1 / x$.
- Normalize (`StandardScaler` no scikit-learn) **apenas as colunas** `MYCT` (transformada), `MMIN`, `MMAX`, `CACH`, `CHMIN`, `CHMAX`.
- Depois, construa *features* polinomiais (`PolynomialFeatures` no scikit-learn) de grau 2 **apenas para as colunas** `MYCT` (transformada), `MMIN`, `MMAX`, `CACH`, `CHMIN`, `CHMAX`.

Não é obrigatório o uso de `Pipeline` ou `ColumnTransformer` - esteja livre para usá-los ou não, conforme preferir.

Aplique esta função para processar os dados de treinamento.

**Questão 3**: Analise o desempenho RMSE de um modelo de regressão linear para estes novos dados usando validação cruzada com 5 partições.

**Questão 4**: Compare os dois modelos investigados nas questões anteriores. Existe evidência de que o modelo mais complexo é melhor que o modelo simples?

**Questão 5**: Retreine o modelo escolhido (se não houver evidencia de superioridade de um modelo sobre o outro, escolha o que você quiser) no conjunto de treinamento completo e meça o desempenho final no conjunto de teste. (O modelo continua sendo uma regressão linear, o que muda é o uso do conjunto de treinamento original ou o com *feature engineering*)

**Questão 6**: Compare o desempenho do seu modelo preditivo com o desempenho do modelo preditivo dos autores do *dataset*, de acordo com as predições vistas na coluna `ERP`. 

**Questão 7**: Quais são os *outliers* do seu modelo preditivo? Ou seja: para quais CPUs no conjunto de teste temos o maior erro, em valores absolutos, na direção positiva e na direção negativa do erro?

Dica: ao descobrir o índice dos outliers, retorne ao conjunto de dados original e descubra a marca e modelo da CPU.


## Parte conceitual

**Questão 8**: Perguntei ao "ChatGPT" o seguinte: "No contexto de machine learning, explique com suas palavras e em menos de 100 palavras os conceitos de sensibilidade e especificidade.". A resposta foi:

"Sensibilidade (ou recall) e especificidade são métricas de desempenho em modelos de machine learning para classificação binária.

Sensibilidade mede a capacidade do modelo em identificar corretamente todos os casos positivos, ou seja, a proporção de verdadeiros positivos (amostras positivas classificadas corretamente) em relação ao total de casos positivos.

Especificidade, por outro lado, avalia a capacidade do modelo em identificar corretamente todos os casos negativos, ou seja, a proporção de verdadeiros negativos (amostras negativas classificadas corretamente) em relação ao total de casos negativos.

Ambas as métricas são importantes para avaliar o desempenho do modelo em diferentes contextos, dependendo das necessidades do problema em questão."

Construa um exemplo do seu cotidiano como aluno do Insper em que o trade-off sensibilidade versus especificidade poderia ser demonstrado. Você não deve incluir exemplos de projetos ou aplicativos de machine learning, ou provas academicas em geral.

Resposta:

**Questão 9**: Perguntei ao "ChatGPT" o seguinte: "Explique a curva ROC para um padeiro". A resposta foi:

"Claro! Vamos adaptar a explicação da curva ROC (Receiver Operating Characteristic) para o contexto de um padeiro.

Imagine que um padeiro está tentando determinar a qualidade de duas marcas de farinha (Marca A e Marca B) para fazer seu pão. Ele quer escolher a farinha que produz o pão mais saboroso.

A curva ROC pode ser comparada à capacidade do padeiro de distinguir entre as duas marcas de farinha. Neste caso:

- Ele testa ambas as farinhas e avalia a qualidade de seus pães.
- Ele varia a quantidade de farinha de cada marca nas receitas para observar como isso afeta a qualidade do pão.
- Em seguida, ele cria uma curva ROC, onde o eixo X representa a quantidade de farinha da Marca A usada na receita e o eixo Y representa a quantidade da Marca B.
- Cada ponto na curva ROC representa uma combinação diferente de quantidades de farinha, e a curva mostra quão bem ele pode distinguir a qualidade dos pães feitos com as duas marcas.
- A área sob a curva ROC (AUC) indica o quão eficaz o padeiro é em diferenciar as farinhas; quanto maior a AUC, melhor a capacidade de escolha.

Assim, a curva ROC ajuda o padeiro a tomar uma decisão informada sobre qual farinha usar para fazer pães deliciosos com base na quantidade de cada marca em sua receita."

Ou seja, a resposta não faz o menor sentido.

Construa uma explicação melhor de curva ROC para um padeiro, que não sabe nada de *machine learning* mas sabe conduzir os negócios de uma padaria com muito sucesso.

Resposta:

**Questão 10**: Construa um exemplo concreto de como o machine learning pode ajudar a aumentar a eficiência do trânsito de São Paulo. O exemplo deve ser específico de *machine learning*, e não de inteligência artificial no sentido mais amplo.

Resposta: